## Reddit Data Pipeline QC

Purpose: Explore methods to filter and clean subreddit data to create a meaningful DIY/home improvement dataset for Workbench RAG pipeline.

## Topics
1. Fetch subreddit posts by flair
2. Filter posts by keywords
3. Define minimum viable fetch posts function for data pipeline
4. Next Steps

In [1]:
import os
from random import sample 
import praw
import keyring 
from praw.models import user
import time

# Reddit API Auth using Keychain Creds 
client_id = keyring.get_password("reddit-client-id", "reddit-api")
client_secret = keyring.get_password("reddit-client-secret", "reddit-api")
user_agent = "TestScript/1.0 by /u/chippetto90"

# Initialize Reddit client
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent,
)

print("User agent:", user_agent)

User agent: TestScript/1.0 by /u/chippetto90


In [3]:

def fetch_posts(reddit, limit=5):
    """Fetch top posts from r/diy subreddit with rate limiting."""
    print(f"Fetching top {limit} posts from r/diy...")
    posts_list = []
    subreddit = reddit.subreddit("diy")
    
    for i, submission in enumerate(subreddit.top(time_filter="year", limit=limit)):
        posts_list.append(submission)
        print(f"Post {i + 1} title: {submission.title}")  # <- added line
        if (i + 1) % 100 == 0:
            print(f"Fetched {i + 1} posts...")
        time.sleep(1.2)  # Respect Reddit API limits
    
    print(f"Successfully fetched {len(posts_list)} posts")
    return posts_list

fetch_posts(reddit)

Fetching top 5 posts from r/diy...
Post 1 title: My wife went to a work event for a few days, in my hubris I thought I could build her a new studio before she got back...
Post 2 title: My Christmas present to my wife this year was renovating our laundry room. How did I do?
Post 3 title: Wife put me in charge of wedding favors and I needed a project.
Post 4 title: Made a bulldozer bed for my 2yo
Post 5 title: 4 months ago I lost all respect for local contractors. 4 months later I've gained a ton for myself.
Successfully fetched 5 posts


[Submission(id='1jjxjn6'),
 Submission(id='1homo1x'),
 Submission(id='1h4kt3o'),
 Submission(id='1lfa89a'),
 Submission(id='1nvc72e')]

In [17]:
#Fetch posts by flair and title keywords 

def fetch_flair_posts(
    reddit, 
    subreddit_name="diy", 
    flair=None, 
    limit=10, 
    sample_size=50, 
    keywords=None,
    delay=0.5 #default sleep between requests
):
    """
    Fetch top posts from a subreddit filtered by flair and optional title keywords.
    
    Parameters:
    - reddit: authenticated PRAW Reddit instance
    - subreddit_name: subreddit to fetch from (default 'diy')
    - flair: text of the flair to filter by (case-insensitive)
    - limit: max number of posts to return
    - sample_size: number of top posts to sample from
    - keywords: list of keywords to filter titles (case-insensitive)
    
    Returns:
    - List of PRAW Submission objects
    """
    posts_list = []
    subreddit = reddit.subreddit(subreddit_name)
    
    if keywords is None:
        keywords = []  # empty list means no keyword filtering

    print(f"Fetching up to {limit} posts from {subreddit_name} with flair: {flair}. Sample size = {sample_size} top posts...\n")
    
    for submission in subreddit.top(time_filter="year", limit=sample_size):
        title_lower = submission.title.lower()
        flair_match = (flair is None) or (submission.link_flair_text and submission.link_flair_text.lower() == flair.lower())
        keyword_match = (not keywords) or any(kw.lower() in title_lower for kw in keywords)
        
        if flair_match and keyword_match:
            posts_list.append(submission)
            if len(posts_list) >= limit:
                break
        time.sleep(delay)  # Respect Reddit API rate limits

    print(f"\nDone! Collected {len(posts_list)} posts.\n")
    return posts_list


In [20]:
#flair= "home improvement" 
#Fetch r/diy posts with flair: home improvement

posts = fetch_flair_posts(
    reddit, 
    flair="home improvement",
    keywords=None,
    limit=10, 
    sample_size=100,
    delay=0.3
)

for post in posts: 
     print(f"- {post.title[:80]} | Flair: {post.link_flair_text}")

Fetching up to 10 posts from diy with flair: home improvement. Sample size = 100 top posts...


Done! Collected 10 posts.

- My wife went to a work event for a few days, in my hubris I thought I could buil | Flair: home improvement
- My Christmas present to my wife this year was renovating our laundry room. How d | Flair: home improvement
- Finished the 2nd floor of my garage into a Home Theatre and Gym. | Flair: home improvement
- My VHS collection got too big so I decided to build my own Blockbuster inside of | Flair: home improvement
- Only took all summer… finally stained my garage door. | Flair: home improvement
- New laundry nook in our 150yr old house! After/Before and Full Process | Flair: home improvement
- Upcycled some bowling alley lanes for flooring in my renovation project! | Flair: home improvement
- 2 days and $200 later, a quick guest bathroom makeover on our 1927 home. | Flair: home improvement
- Laid a full wall of herringbone tile wrong. Now what? | Flair: home impr

In [21]:
#flair= "help"
#Fetch r/diy posts with flair: help 

posts = fetch_flair_posts(
    reddit, 
    flair="help",
    keywords=None,
    limit=10, 
    sample_size=100
)

for post in posts: 
     print(f"- {post.title[:80]} | Flair: {post.link_flair_text}")


Fetching up to 10 posts from diy with flair: help. Sample size = 100 top posts...


Done! Collected 5 posts.

- Not sure if this is the right audience | Flair: help
- We had wedding guests sign a whiskey barrel as a memento. What's the best way to | Flair: help
- Duct covering | Flair: help
- How bad is this? | Flair: help
- Update: Laid a full wall of herringbone tile wrong. Now what? | Flair: help


In [ ]:
#Minimal Keyword Filtering Test 
#See how many posts in given sample of subreddit contain a given keyword. 

subreddit = reddit.subreddit("diy")
sample_posts = list(subreddit.top(time_filter="year", limit=20))  # small sample

#Test Keyword: 
keywords = ["my"]  

matches = []
for post in sample_posts:
    match = any(kw.lower() in post.title.lower() for kw in keywords)
    print(f"Title: {post.title[:60]} | Match: {match}")  # debug each post
    if match:
        matches.append(post)

print("\nPosts with Keyword Match:")
for post in matches:
    print(f"- {post.title[:80]} | Flair: {post.link_flair_text}")

In [ ]:

#Keyword Filtering Test with Stats

subreddit = reddit.subreddit("diy")
sample_posts = list(subreddit.top(time_filter="year", limit=20))  # small sample for testing

# Keywords to Test: 
keywords = ["my"]  # just to see if filtering works

# Track matches
matches = []

for post in sample_posts:
    match = any(kw.lower() in post.title.lower() for kw in keywords)
    matches.append(match)
    print(f"Title: {post.title[:60]} | Match: {match}")

# Calculate stats
total = len(matches)
true_count = sum(matches)
percent_true = true_count / total * 100 if total else 0

print("\nSummary:")
print(f"Total posts sampled: {total}")
print(f"Posts matching keyword: {true_count}")
print(f"Percent matching: {percent_true:.1f}%")

Title: My wife went to a work event for a few days, in my hubris I  | Match: True
Title: My Christmas present to my wife this year was renovating our | Match: True
Title: Wife put me in charge of wedding favors and I needed a proje | Match: False
Title: Made a bulldozer bed for my 2yo | Match: True
Title: 4 months ago I lost all respect for local contractors. 4 mon | Match: True
Title: Rental Remodel After Nightmare Tenant  | Match: False
Title: Not sure if this is the right audience | Match: False
Title: Wife said I should just use wood filler, I had a better idea | Match: False
Title: Was tired of my janky walkout - added deck tiles and paint | Match: True
Title: Finished the 2nd floor of my garage into a Home Theatre and  | Match: True
Title: My VHS collection got too big so I decided to build my own B | Match: True
Title: She Shed build for the wife in 2021 | Match: False
Title: Only took all summer… finally stained my garage door. | Match: True
Title: Update: 12ft Wendy’s Sign in 

## Results of Flair Only Tests

- **Home Improvement Flair (Top 100 posts)**  
  - Returned 10 posts, but most were project showcases rather than instructional/how-to content.  

- **Help Flair (Top 100 posts)**  
  - Returned 5 posts.  
  - Some posts were actual questions or "how-to" style, others were situational/context-based.  

**Key Takeaways:**  
- Flair-only filtering works but **does not guarantee instructional content**.  
- Some flairs (like "help") are **sparse in top posts**, limiting relevant content.  
- To improve recall, consider **combining flair + keyword search in title and body** or using **PRAW search queries** directly.


## MVF - Minimum Viable Fetch for MVP

To get a working end-to-end RAG pipeline sooner rather than later, we define a **minimal fetch function**.  

This function acts as a **placeholder** for MVP development allowing us to ingest posts, embed text w/ OpenAI embeddings API, implement vector storage, implement retrieval, and **test the full pipeline** without over-optimizing for data quality first. 

### Minimum Viable Fetch Requirements

- Fetches top posts from a subreddit, optionally filtered by flair. 

- Designed to be **fast and simple**: no keyword or body filtering yet. 
  
- Add print_titles flag - quickly see what posts were fetched w/o looping through manually everytime. 

 ``` python
        if print_titles:
        print(f"- {submission.title[:80]} | Flair: {submission.link_flair_text}")
```
- Return a simple summary: Like {"posts": posts_list, "count": len(posts_list)}. Easier to log stats in testing/notebook. 

- Parameterize `time_filter` - defaulting to `year`, but need to extend to allow `all`, `month`, etc. 
- Rate-limiting parameter - make the `delay` adjustable per call so we can speed up or slow down without editing function itself everytime
- Update docstring to note that this is the placeholder fetch for this mvp. 


In [ ]:
#Minimum Viable Fetch Function 

def fetch_mvp_posts(
    reddit, 
    subreddit_name="diy", 
    flair=None, 
    limit=10, 
    sample_size=50, 
    delay=0.5
):
    """
    Minimal fetch for MVP: grab top posts with optional flair.
    No keyword filtering yet — we just want enough posts to test RAG pipeline.
    
    Parameters:
    - reddit: authenticated PRAW instance
    - subreddit_name: str
    - flair: str or None (optional)
    - limit: max posts to return
    - sample_size: top posts to sample
    - delay: seconds to sleep between API calls
    """
    posts_list = []
    subreddit = reddit.subreddit(subreddit_name)
    
    for submission in subreddit.top(time_filter="year", limit=sample_size):
        if flair is None or (submission.link_flair_text and submission.link_flair_text.lower() == flair.lower()):
            posts_list.append(submission)
            if len(posts_list) >= limit:
                break
        time.sleep(delay)
    
    print(f"Fetched {len(posts_list)} posts from r/{subreddit_name} with flair={flair}")
    return posts_list

# Example call for MVP
mvp_posts = fetch_mvp_posts(reddit, flair="help", limit=10, sample_size=50)


In [ ]:
#Example Minimum Viable Function Call: 

# Minimal example, flair only
results = fetch_mvp_posts(reddit, flair="home improvement", limit=5, sample_size=50)

# Access posts list
posts = results["posts"]
print("Number of posts collected:", results["count"])

### 💡 Next Steps After MVP

- Add a Keyword stats wrapper function: count matches, calculate percentages.
- Add Text/body filtering: include post body for richer “how-to” detection.
- Define modular data eval framework: to determine best data filtering methods to product high quality dataset for future scaling
- RAG integration: feed these posts into embeddings → vector DB → retrieval pipeline.

#### Example: Lightweight keyword stats wrapper

Works with `fetch_posts_mvp()` output. Can be extended later with more keywords, flair combos, more complex rules

In [ ]:
def keyword_stats(posts_dict, keywords):
    """
    Evaluate posts against a list of keywords in their titles.

    Parameters:
    - posts_dict: dict returned by fetch_mvp_posts (must have 'posts' list)
    - keywords: list of strings to search for in post titles (case-insensitive)

    Returns:
    - dict with:
        - total_posts: int, total number of posts checked
        - matches: int, number of posts containing at least one keyword
        - match_pct: float, percentage of posts matching keywords
        - matched_titles: list of matched post titles
    """
    posts = posts_dict.get("posts", [])
    matched_titles = []

    for post in posts:
        title_lower = post.title.lower()
        if any(kw.lower() in title_lower for kw in keywords):
            matched_titles.append(post.title)

    total_posts = len(posts)
    matches = len(matched_titles)
    match_pct = (matches / total_posts * 100) if total_posts else 0.0

    print(f"\nKeyword stats:")
    print(f"Total posts checked: {total_posts}")
    print(f"Posts matching keywords {keywords}: {matches}")
    print(f"Percentage match: {match_pct:.1f}%\n")
    
    if matched_titles:
        print("Matched post titles:")
        for t in matched_titles:
            print(f"- {t[:80]}...")  # truncate for notebook display

    return {
        "total_posts": total_posts,
        "matches": matches,
        "match_pct": match_pct,
        "matched_titles": matched_titles
    }